# Init

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import pandas as pd
import h5py
import numpy as np

In [2]:
X_test = h5py.File("/Users/tdeborde/goinfre/X_test_eFVIB85.h5","r")

def get_train():
    X_train = h5py.File("/Users/tdeborde/goinfre/X_train_KBHhQ0d.h5","r")
    y_train = pd.read_csv("/Users/tdeborde/goinfre/y_train_2.csv","r", delimiter=',')

    X_train = X_train["features"][:]
    return X_train, y_train

In [5]:
X_train, y_train = get_train()

# Features

In [9]:
def extract_features(data):
    features = []
    features.append(data[:, :11])
    features.append(data[:, 11:].max(1).reshape(-1, 1))
    features.append(data[:, 11:].min(1).reshape(-1, 1))
    features.append(np.abs(data[:, 11:]).mean(1).reshape(-1, 1))
    features = np.concatenate(features, 1)
    return features

def extract_manual(data):
    X = data[:, :11]
    features = np.concatenate((
        np.mean(X, axis=1)[..., np.newaxis],
        np.amin(X, axis=1)[..., np.newaxis],
        np.amax(X, axis=1)[..., np.newaxis]
    ), axis=1)
    quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    for q in quantiles:
        features = np.concatenate((features, np.quantile(X, q, axis=1)[..., np.newaxis]), axis=1)
    X = np.concatenate((data[:, :11], features), axis=1)
    return X

In [10]:
X_train = extract_features(X_train)
X, X_val, y, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Train

In [ ]:
clf = RandomForestClassifier(n_estimators=10, verbose=3, n_jobs=-1, random_state=42)
clf.fit(X, y)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 10building tree 2 of 10
building tree 3 of 10
building tree 4 of 10



In [24]:
y_val.shape

(65409, 2)

# Test

In [ ]:
features_test = extract_features(X_test)
y_pred = clf.predict(features_test)
with open("y_benchmark.csv", "w") as f:
    f.write("".join(["id,label\n"] + ["{},{}\n".format(i, y) for i, y in enumerate(y_pred)]))